#  Information

Table Description: "EARNINGS IN THE PAST 12 MONTHS (IN 2018 INFLATION-ADJUSTED DOLLARS)" - American Community Service (ACS), TableID S2001.

Download from [data.census.org](https://data.census.gov/cedsci/table?q=ACSST5Y2018.S2001&g=0100000US.860000_1600000US3651000&y=2018&tid=ACSST5Y2018.S2001&moe=false&hidePreview=true).

We use the 5 year estimates since the 1 year estimates do not have enough granularity for geo analysis (no zip code). Notice that that even though it consider the past 5 years to make the estimate, the estimate is still of the **earning in the past 12 months**.

We use the data from 2018 instead of 2019 since 2019 does not have a 5 year estimates so again no zip codes.

See also [When to Use 1-year, 3-year, or 5-year Estimates](https://www.census.gov/programs-surveys/acs/guidance/estimates.html) for more information about the data acquisition methodology.

Notice that we select all zip codes and filter the NY ones since we can't manually select them in the tool (selecting new york state will select that as a row and not filter the results). 
Notice also that the last row in the link is just "New York City" as a aggregate so that we can check if our zip code selection seems correct.



In [204]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
import seaborn as sns
sns.set()
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
DATA_DIR  = "../data/"
#FILE_NAME = "ACSST5Y2018.S2001_data_with_overlays_2020-11-20T230734.csv"
FILE_NAME = "ACSST5Y2018.S2001_data_with_overlays_2020-11-20T230734.csv"
FILE_METADATA = FILE_NAME.replace("data_with_overlays", "metadata")

In [108]:
# Uncomment this line to check out the main data columns in the file:
#!cat $DATA_DIR$FILE_METADATA | grep -E -i -v "male|margin" 

In [106]:
rename = {
    "GEO_ID": "geo",
    "NAME": "zcta",
    # "S2001_C01_002E","Estimate!!Total!!
    # Population 16 years and over with earnings!!Median earnings (dollars)"
    "S2001_C01_002E": "median_earning",
    "S2001_C01_013E": "full_time_median",
    "S2001_C01_014E": "full_time_mean",
}

In [152]:
interest_cols = list(rename.values())
# The second row is just the metadata info as string, so we can skip
income = pd.read_csv(DATA_DIR + FILE_NAME, skiprows=[1])  
income.rename(columns=rename, inplace=True)
income = income.loc[:, interest_cols]

# remove last row and save as NY city
new_york_city_total = income.iloc[-1]
income.drop(index=[len(income)-1], inplace=True)
income.head(5)

,geo,zcta,median_earning,full_time_median,full_time_mean
0,8600000US00601,ZCTA5 00601,14448,19224,24059
1,8600000US00602,ZCTA5 00602,13322,19424,26194
2,8600000US00603,ZCTA5 00603,15980,24351,36881
3,8600000US00606,ZCTA5 00606,10554,17352,19420
4,8600000US00610,ZCTA5 00610,15298,20174,25518


In [153]:
# Make sure all ZCTA fields start the same and remove prefic
(income["zcta"].str.slice(0, 6) != "ZCTA5 ").sum()
income["zcta"] = income["zcta"].str.slice(6) # use 6 bcs of extra space
income["zcta"] = pd.to_numeric(income["zcta"], errors="coerce")
assert income["zcta"].isna().sum() == 0

Let's work with just NYC data;

Analyzing the ZCTA to ZIP using [NYC ZIP Code to ZCTA Crosswalk](http://faculty.baruch.cuny.edu/geoportal/resources/nyc_geog/zip_to_zcta10_nyc_revised.xls)
from [Baruch College](https://www.baruch.cuny.edu/confluence/display/geoportal/NYC+Geographies)
, we see that various zips map to the sama zcta, so we can't convert ZCTA -> ZIP (only the other way);
Therefore, we will assume our ZIP is just the ZCTA in all cases.

In [154]:
ZIP_FILE = "zip_to_zcta10_nyc_revised.csv"
zip_to_zcta = pd.read_csv(DATA_DIR + ZIP_FILE)
# Different zip, same zcta5
zip_to_zcta.head(2)

,zipcode,ziptype,postalcity,zcta5,bcode,note
0,10001,ZIP Code area,New York,10001,36061,NaN
1,10118,Post Office or large volume customer,New York,10001,36061,NaN


In [190]:
zip_lost = zip_to_zcta["zipcode"].nunique() - zip_to_zcta["zcta5"].nunique()
print("Amount of zip codes we lose in the covnersion",
      f"{zip_lost} = {100*zip_lost/zip_to_zcta['zipcode'].nunique():.2f}%")

Amount of zip codes we lose in the covnersion 103 = 32.49%


In [219]:
all_ny_zcta = zip_to_zcta["zcta5"].unique()
ny_city_income = income.query("zcta in @all_ny_zcta").copy(deep=True)

In [221]:
for focus_col in ["median_earning", "full_time_median", "full_time_mean"]:
    invalid_earnings = pd.to_numeric(ny_city_income[focus_col], errors="coerce").isna()
    print(focus_col, f"% of invalid earning: {100*invalid_earnings.mean():.2f}%")
    print("Invalid option:", ny_city_income.loc[invalid_earnings, focus_col].unique())
# Since all columns have the same amount of missing values, let's go with median
focus_col = "median_earning"

median_earning % of invalid earning: 0.00%
Invalid option: []
full_time_median % of invalid earning: 0.00%
Invalid option: []
full_time_mean % of invalid earning: 0.55%
Invalid option: ['N']


In [224]:
ny_city_income[focus_col] = pd.to_numeric(ny_city_income[focus_col],
                                          errors="coerce")
ny_city_income.dropna(subset=[focus_col], inplace=True)
ny_city_income.head(2)

,geo,zcta,median_earning,full_time_median,full_time_mean
2558,8600000US10001,10001,74878.0,93452,136402
2559,8600000US10002,10002,37348.0,55285,75269


In [229]:
"""As a ballpark estimate, the median of the median should be aprox the global 
median we orginally had from the data. We don't expect it to match up exactly 
since we aren't considering the different pop sizes for each zip code.
"""
ny_city_income[focus_col].median(), new_york_city_total[focus_col]

(39479.0, '38845')

# Merging with complaints

In [249]:
FILE_SAMPLE = "sample_complaints.csv"
# Merge with complaints
sample_complaints = pd.read_csv(DATA_DIR + FILE_SAMPLE)
sample_complaints.head(2)

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,BBL,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Park Facility Name,Park Borough,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,25356119,04/13/2013 03:55:00 PM,04/13/2013 03:55:00 PM,DEP,Department of Environmental Protection,Sewer,Sewer Backup (Use Comments) (SA),NaN,11429.0,215-27 HOLLIS AVENUE,HOLLIS AVENUE,215 ST,216 ST,NaN,NaN,ADDRESS,Queens Village,NaN,NaN,Closed,NaN,The Department of Environmental Protection has...,04/13/2013 03:55:00 PM,13 QUEENS,4.111010e+09,QUEENS,1055561.0,198095.0,PHONE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.710114,-73.742781,"(40.7101144, -73.7427809)"
1,25356123,04/13/2013 03:36:00 PM,04/13/2013 04:00:00 PM,DEP,Department of Environmental Protection,Water System,Hydrant Leaking (WC1),NaN,10460.0,EAST TREMONT AVENUE,EAST TREMONT AVENUE,PROSPECT AVENUE,MAPES AVENUE,NaN,NaN,BLOCKFACE,BRONX,NaN,NaN,Closed,NaN,The Department of Environmental Protection det...,04/13/2013 04:00:00 PM,06 BRONX,NaN,BRONX,1015108.0,246825.0,PHONE,Unspecified,BRONX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.844100,-73.888469,"(40.8440997, -73.8884686)"


In [250]:
print("% ofinvalid zips", 100*sample_complaints["Incident Zip"].isna().mean())
sample_complaints.dropna(subset=["Incident Zip"], inplace=True)
sample_complaints["processed_zip"] = sample_complaints["Incident Zip"].astype(int)

% ofinvalid zips 0.40040040040040037


In [260]:
merged = pd.merge(sample_complaints, ny_city_income, left_on="processed_zip", 
         right_on="zcta", validate="m:1"
)
merged[["zcta", focus_col]].head(2)

,zcta,median_earning
0,11429,35145.0
1,11429,35145.0


In [298]:
import plotly.express as px
# Quick and dirty viz; I should be mapping zip to geoJson, but ok for now;
# This ignores 40% of data
temp = merged.dropna(subset=["Latitude", "Longitude"])  

# Requires a mapbox token
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(temp, lat="Latitude", lon="Longitude", color="median_earning",
                    size_max=15, zoom=9)
#fig

In [287]:
len(temp) / len(sample_complaints)

0.6040201005025125

In [291]:
fig.write_image("1.1-Income_analysis.png")

![](1.1-Income_analysis.png)

In [295]:
%%javascript
$('#menubar').toggle();

<IPython.core.display.Javascript object>